In [549]:
import pandas as pd
import pymatreader as pmr
import numpy as np

In [550]:
file_name='S01' #Указать номер человека#
data=pmr.read_mat(f'исходное/{file_name}.mat') #Указать путь к файлу#
df_not_clean=pd.DataFrame(data[file_name])

In [551]:
useless_demographic=['Age', 'Gender', 'DominantHand', 'Education', 'MaritalStatus', 'Children']
useless_profile=['WeeklySupermarketVisits', 'SupermarketVisitDuration', 'PriceImpact', 'BrandImpact', 'DiscountImpact', 'AdvertisementImpact', 'SuggestionImpact', 'ShoppingList', 'VerbalVisual', 'Spontaneous', 'VarietySeeker', 'UtilitarianMotivation', 'HedonicMotivation', 'Extraversion', 'Neuroticism', 'Agreeableness', 'Openness', 'Conscientiousness', 'BargainHunter']

In [552]:
df_not_clean.rename(index={
    'education': 'Education',
    'marital_status': 'MaritalStatus'
}, inplace=True)

In [553]:
useless_products=np.arange(1, 25, 1)
s=[]
for i in useless_products:
    s.append('Product'+str(i))
useless_products=s

In [554]:
eeg_useful_data=df_not_clean['EEG_clean']
eeg_useful_data=eeg_useful_data.drop(useless_demographic+useless_profile+useless_products)
eeg_useful_data=pd.DataFrame(eeg_useful_data).transpose()

In [555]:
eeg_data=eeg_useful_data.Data['EEG_clean']
eeg_chans=eeg_useful_data.chans['EEG_clean']
electrodes=eeg_chans['labels']
df_products=df_not_clean.drop(columns=['Profile', 'Demographics', 'EEG_clean', 'ET_clean'])
df_products.drop(useless_demographic+useless_profile+['Data', 'Fs', 'chans'], inplace=True)
useless_pages=['Page1', 'Page2', 'Page3', 'Page4', 'Page5', 'Page6']
num_of_products=24

In [556]:
df_eeg=pd.DataFrame()

In [ ]:
#Как-будто не работает#
for t in useless_pages:
    for q in range(24):
        prdct=df_products[t][q]
        eeg_vrem_seg=prdct['EEG_segments']
        for i in eeg_vrem_seg:
            if isinstance(i, np.int32):
                eeg_vrem_seg=np.delete(eeg_vrem_seg, np.where(eeg_vrem_seg==i)) #?#

In [ ]:
for t in useless_pages:
    for q in range(num_of_products):
        
        dfvrem_eeg=pd.DataFrame()
        prodct=df_products[t][q]
        product_info=prodct['ProductInfo']
        eeg_prodct=prodct['EEG_segments']
        
        if len(eeg_prodct)==0:
            continue
        else:

            for i in electrodes:
                elec_index=eeg_chans['labels'].index(i)
                eeg_i=eeg_data[elec_index]
                eeg_i_for_prodct=[]

                if isinstance(eeg_prodct[0], np.ndarray):
                    for j in eeg_prodct:
                        s=[]
                        for k in range(j[1]-j[0]+1): #ПРОВЕРИТЬ НА СЧЕТ +1##
                            s.append(eeg_i[j[0]+k]-1)
                        eeg_i_for_prodct.append(s)

                else:
                    s=[]
                    for k in range(eeg_prodct[1]-eeg_prodct[0]+1):
                        s.append(eeg_i[eeg_prodct[0]+k]-1)
                    eeg_i_for_prodct.append(s)
                dfvrem_eeg[f'{electrodes[elec_index]}']=[eeg_i_for_prodct]

            dfvrem_eeg['info']=[product_info]
            dfvrem_eeg['product_page']=t
            dfvrem_eeg['product_number']=q
            df_eeg=pd.concat([df_eeg, dfvrem_eeg], ignore_index=True)
df_eeg.reset_index(inplace=True)

In [559]:
df_eeg

,index,P3,C3,F3,Fz,F4,C4,P4,Cz,Pz,...,T5,O1,O2,F7,F8,T6,T4,info,product_page,product_number
0,0,"[[6.511624565954574, 6.890339681978484, 7.0479...","[[1.9735072833038174, 1.6912671651119324, 1.08...","[[-13.663424789553144, -13.288257822583109, -1...","[[-1.2626322975512119, -1.0850765316761237, -0...","[[-3.4447362739840486, -3.4761841312343846, -3...","[[-0.9215066908380872, -0.728643615814696, -0....","[[3.8405385820332985, 4.49471437337068, 5.0971...","[[0.8548347410662942, 0.9164246061992112, 1.00...","[[12.118382198416224, 11.884920570144109, 11.3...",...,"[[9.948028512453027, 10.167699071244604, 10.07...","[[11.198039807872737, 11.743789693020632, 11.8...","[[5.089997682116887, 6.183077571677555, 7.5033...","[[-24.71970508052686, -24.55931740480053, -24....","[[-0.4369746731655312, 0.43927261797395634, 1....","[[6.133394595485935, 7.30839794434627, 8.59491...","[[5.250415460704896, 5.170446107650037, 4.9410...","{'Bought': 0, 'Reasons': [], 'Familiarity': 0,...",Page1,0
1,1,"[[-1.2549047519936847, 0.9134356856991255, 3.2...","[[-1.4202792253378398, -0.19254157221640045, 1...","[[-7.977386435582023, -9.551373698918491, -11....","[[4.355951106464545, 3.1867315975663857, 1.537...","[[0.8754074712434561, 0.09599189923910911, -1....","[[-1.1522636571871994, -0.9586634443896376, -0...","[[-0.3310030143261543, 0.34022443459226337, 1....","[[4.340729889230897, 3.8035989835392403, 2.972...","[[-0.5393957945377332, 2.921494987479329, 7.15...",...,"[[1.158547483225734, 3.6640470784569006, 6.501...","[[-0.2689594269643193, 2.82631685532524, 6.497...","[[2.5096020664309804, 2.731405452979201, 2.932...","[[-16.819620679774964, -18.997273225855817, -2...","[[4.961607184266291, 3.587475439508866, 1.5576...","[[6.723283866269348, 6.458234307591, 5.7024098...","[[4.570872852472049, 4.987055555651538, 5.2633...","{'Bought': 0, 'Reasons': [], 'Familiarity': 0,...",Page1,1
2,2,"[[13.121708433187555, 12.509060487302973, 13.4...","[[7.688418223173944, 7.457237092855296, 7.9794...","[[4.381839914005072, 3.047007147929164, 2.9467...","[[5.431037100045795, 3.1510225868231103, 2.058...","[[0.728675542684567, -0.853773563541658, -1.88...","[[0.929985415993761, -1.455964982220128, -0.87...","[[6.521799141979645, 7.132231722024411, 8.4150...","[[6.533582197513108, 4.811115542453948, 6.4638...","[[5.261974850470187, 5.709249542323662, 5.7491...",...,"[[8.877344445299023, 7.495693646941758, 7.3020...","[[4.77346720674749, 4.884057757775336, 5.29520...","[[12.747592874526902, 14.568933736992738, 16.3...","[[-3.5681245551291854, -3.677996229061066, -2....","[[-5.45294404058336, -6.15102723281231, -5.270...","[[-1.2861775908599005, -3.2786071068437592, -0...","[[-8.367057299627945, -9.799016106320456, -8.6...","{'Bought': 0, 'Reasons': [], 'Familiarity': 0,...",Page1,2
3,3,"[[0.7586719793308871, 3.7059941635699083, 5.98...","[[6.9197916370651615, 9.506495014560233, 10.87...","[[13.536285892561292, 14.547366696026963, 14.8...","[[1.247407546646285, 3.0548092325195775, 4.490...","[[7.179835385350097, 9.086316947466921, 10.246...","[[7.033802642990743, 8.694582030308384, 9.3932...","[[6.70622966310351, 7.42090588063126, 7.601049...","[[-9.400309187342021, -7.36938186855764, -5.48...","[[11.550085256323856, 12.864619591759615, 13.7...",...,"[[-9.574182809329962, -7.741193993263233, -6.1...","[[3.4769394063968706, 6.028637739230541, 8.608...","[[-1.1153452040663614, 1.3522083738786241, 4.0...","[[19.434026698625743, 19.308095645411694, 19.3...","[[2.4658919840628557, 2.317926214264704, 1.702...","[[0.19499160188490672, 1.8647258897374668, 3.4...","[[1.7949760936222345, 3.276051333791316, 4.121...","{'Bought': 1, 'Reasons': ['discount', 'combina...",Page1,3
4,4,"[[0.8223175204848647, 0.4718761352087997, -0.3...","[[5.198472496217983, 4.010929923811551, 2.2928...","[[-0.5022978608320713, -1.7358646143129453, -3...","[[8.565720072529816, 8.501227898735818, 7.5435...","[[3.9582125193991384, 3.6352588948667854, 2.94...","[[-1.4600043761086745, -1.3833847455751593,

In [560]:
#df_eeg.to_csv(f'eeg_{file_name}.csv')